In [ ]:
# parameters
PROJECT_ID = "nice-proposal-467718-q6"
REGION = "us-west1"
BRONZE_PATH = "gs://meu-bucket-premier/bronze/"
SILVER_PATH = "gs://meu-bucket-premier/silver/"
GOLD_PATH = "gs://meu-bucket-premier/gold/"
RUN_TS = None


## Camada Gold – matches_2014_2020

### Objetivo
Transformar os dados tratados da camada **Silver** em uma versão final e pronta para consumo analítico, mantendo apenas informações essenciais e aplicando regras de negócio.

### Fonte de Dados
- **Origem:** `gs://meu-bucket-premier/silver/epl/matches_2014_2020/matches_2014_2020.csv`
- **Camada anterior:** Silver (`matches_2014_2020_silver`)

### Tratamentos Aplicados
1. **Seleção de colunas relevantes** para análises e dashboards.
2. **Padronização de tipos de dados** (datas, numéricos e textos).
3. **Remoção de registros inválidos**:
   - Partidas com ano igual a 0.
   - Linhas com informações essenciais ausentes.
4. **Mascaramento de informações sensíveis**:
   - Inclusão fictícia de **CNPJ** para clubes.
   - Inclusão fictícia de **CPF** para jogadores.

### Resultado
- Dados prontos para análise final.
- Estrutura otimizada para consumo direto em ferramentas de BI e modelos de Machine Learning.
- Armazenamento em **Parquet** na camada Gold para melhor desempenho.

📍 **Caminho final:**  
`gs://meu-bucket-premier/gold/epl/matches_2014_2020/matches_2014_2020_gold.parquet`


In [ ]:
%pip install -q pandas gcsfs

import pandas as pd
import numpy as np
import gcsfs
from datetime import datetime, timezone

In [3]:
fs = gcsfs.GCSFileSystem(token="cloud")

In [4]:
# ---- paths ----
BUCKET = "gs://meu-bucket-premier"
SILVER_MATCHES = f"{BUCKET}/silver/epl/matches_2014_2020/matches_2014_2020_silver.csv"
GOLD_SEASON_STATS = f"{BUCKET}/gold/epl/season_team_stats/season_team_stats.csv"

In [5]:
# enriquece com CNPJ mascarado do time se existir
GOLD_TEAMS_WITH_CNPJ = f"{BUCKET}/gold/epl/teams_with_cnpj/teams_with_cnpj.csv"

In [6]:
# ---- ler matches silver ----
matches = pd.read_csv(fs.open(SILVER_MATCHES, "rb"))

In [7]:
matches.head(2)

,Home Team,Away Team,Score,Half Time Score,Match Excitement,Home Team Rating,Away Team Rating,Home Team Possession %,Away Team Possession %,Home Team Off Target Shots,...,Away Team Fouls,Away Team Yellow Cards,Away Team Second Yellow Cards,Away Team Red Cards,Home Team Goals Scored,Away Team Goals Scored,Home Team Goals Conceeded,Away Team Goals Conceeded,year,_dataset
0,MAN UTD,SWANSEA,1-2,0-1,5.9,5.6,7.6,60,40,5.0,...,20.0,4.0,0.0,0.0,1.0,2.0,2.0,1.0,2014,matches_2014_2020
1,WEST BROM,SUNDERLAND,2-2,1-1,7.3,6.5,7.4,58,42,5.0,...,9.0,1.0,0.0,0.0,2.0,2.0,2.0,2.0,2014,matches_2014_2020


In [8]:
# ---- detectar colunas (robusto a variações) ----
def pick(cols, *cands):
    cl = [c.lower() for c in cols]
    # match exato
    for cand in cands:
        if cand.lower() in cl:
            return cols[cl.index(cand.lower())]
    # contém
    for cand in cands:
        for i,c in enumerate(cl):
            if cand.lower() in c:
                return cols[i]
    return None

In [9]:
COL_HOME = pick(matches.columns, "Home Team", "home_team", "HomeTeam", "mandante", "home")
COL_AWAY = pick(matches.columns, "Away Team", "away_team", "AwayTeam", "visitante", "away")
COL_HG   = pick(matches.columns, "Home Team Goals Scored", "FTHG", "home_goals", "gols_mandante")
COL_AG   = pick(matches.columns, "Away Team Goals Scored", "FTAG", "away_goals", "gols_visitante")
COL_YEAR = pick(matches.columns, "year", "season_year", "ano")

In [11]:
missing = [n for n,v in {
    "Home Team":COL_HOME, "Away Team":COL_AWAY, "HG":COL_HG, "AG":COL_AG
}.items() if v is None]
if missing:
    raise ValueError(f"Faltam colunas essenciais em matches: {missing}\nColunas={list(matches.columns)}")

In [12]:
# ---- criar Season (se tiver Year usamos Year-(Year+1); senão tenta por data) ----
if COL_YEAR:
    matches["Season"] = matches[COL_YEAR].astype(int).astype(str) + "-" + (matches[COL_YEAR].astype(int)+1).astype(str)
else:
    COL_DATE = pick(matches.columns, "Date", "match_date", "data", "matchday", "dt")
    if COL_DATE is None:
        raise ValueError("Não achei Year nem Date pra montar Season.")
    dt = pd.to_datetime(matches[COL_DATE], errors="coerce", utc=True)
    matches["Season"] = dt.apply(lambda x: f"{x.year}-{x.year+1}" if x.month>=8 else f"{x.year-1}-{x.year}")

In [13]:
# ---- resultado e placar (H/D/A) ----
matches[COL_HG] = pd.to_numeric(matches[COL_HG], errors="coerce").fillna(0).astype(int)
matches[COL_AG] = pd.to_numeric(matches[COL_AG], errors="coerce").fillna(0).astype(int)

matches["Result"] = np.where(matches[COL_HG] > matches[COL_AG], "H",
                      np.where(matches[COL_HG] < matches[COL_AG], "A", "D"))
matches["FullTimeScore"] = matches[COL_HG].astype(str) + "-" + matches[COL_AG].astype(str)

In [14]:
# ---- agrega stats por time/temporada ----
home = matches.groupby(["Season", COL_HOME]).agg(
    Jogos_Casa    =(COL_HOME, "count"),
    Vitorias_Casa =("Result", lambda x: (x=="H").sum()),
    Empates_Casa  =("Result", lambda x: (x=="D").sum()),
    Derrotas_Casa =("Result", lambda x: (x=="A").sum()),
    Gols_Marc_Casa=(COL_HG, "sum"),
    Gols_Sof_Casa =(COL_AG, "sum"),
).reset_index().rename(columns={COL_HOME:"Team"})

In [15]:
away = matches.groupby(["Season", COL_AWAY]).agg(
    Jogos_Fora    =(COL_AWAY, "count"),
    Vitorias_Fora =("Result", lambda x: (x=="A").sum()),
    Empates_Fora  =("Result", lambda x: (x=="D").sum()),
    Derrotas_Fora =("Result", lambda x: (x=="H").sum()),
    Gols_Marc_Fora=(COL_AG, "sum"),
    Gols_Sof_Fora =(COL_HG, "sum"),
).reset_index().rename(columns={COL_AWAY:"Team"})

In [16]:
stats = home.merge(away, on=["Season","Team"], how="outer").fillna(0)

In [17]:
# totais
stats["Jogos"]      = stats["Jogos_Casa"] + stats["Jogos_Fora"]
stats["Vitorias"]   = stats["Vitorias_Casa"] + stats["Vitorias_Fora"]
stats["Empates"]    = stats["Empates_Casa"] + stats["Empates_Fora"]
stats["Derrotas"]   = stats["Derrotas_Casa"] + stats["Derrotas_Fora"]
stats["Gols_Marc"]  = stats["Gols_Marc_Casa"] + stats["Gols_Marc_Fora"]
stats["Gols_Sof"]   = stats["Gols_Sof_Casa"] + stats["Gols_Sof_Fora"]
stats["Saldo_Gols"] = stats["Gols_Marc"] - stats["Gols_Sof"]
stats["Pontos"]     = stats["Vitorias"]*3 + stats["Empates"]

In [18]:
try:
    teams_with_cnpj = pd.read_csv(fs.open(GOLD_TEAMS_WITH_CNPJ, "rb"))
    # tenta achar colunas de nome e do cnpj mascarado
    col_team_name = pick(teams_with_cnpj.columns, "team", "team_name", "club", "squad")
    col_cnpj_mask = pick(teams_with_cnpj.columns, "cnpj_mascarado", "cnpj_masked")
    if col_team_name and col_cnpj_mask:
        stats = stats.merge(
            teams_with_cnpj[[col_team_name, col_cnpj_mask]].drop_duplicates(),
            left_on="Team", right_on=col_team_name, how="left"
        ).drop(columns=[col_team_name])
        stats.rename(columns={col_cnpj_mask:"Team_CNPJ_Masked"}, inplace=True)
except Exception as e:
    print("Não consegui enriquecer com CNPJ mascarado (seguindo sem):", e)

In [19]:
# ordena ranking
stats = stats.sort_values(["Season","Pontos"], ascending=[True, False]).reset_index(drop=True)
stats["_processing_ts"] = datetime.now(timezone.utc).isoformat(timespec="seconds")

In [21]:
stats.head()

,Season,Team,Jogos_Casa,Vitorias_Casa,Empates_Casa,Derrotas_Casa,Gols_Marc_Casa,Gols_Sof_Casa,Jogos_Fora,Vitorias_Fora,...,Jogos,Vitorias,Empates,Derrotas,Gols_Marc,Gols_Sof,Saldo_Gols,Pontos,Team_CNPJ_Masked,_processing_ts
0,2014-2015,CHELSEA,19,15,4,0,36,9,19,11,...,38,26,9,3,73,32,41,87,NaN,2025-08-13T18:32:36+00:00
1,2014-2015,MAN CITY,19,14,3,2,44,14,19,10,...,38,24,7,7,83,38,45,79,NaN,2025-08-13T18:32:36+00:00
2,2014-2015,ARSENAL,19,12,5,2,41,14,19,10,...,38,22,9,7,71,36,35,75,NaN,2025-08-13T18:32:36+00:00
3,2014-2015,MAN UTD,19,14,2,3,41,15,19,6,...,38,20,10,8,62,37,25,70,NaN,2025-08-13T18:32:36+00:00
4,2014-2015,TOTTENHAM,19,10,3,6,31,24,19,9,...,38,19,7,12,58,53,5,64,NaN,2025-08-13T18:32:36+00:00


In [20]:
# ---- salvar Gold ----
with fs.open(GOLD_SEASON_STATS, "wb") as f:
    stats.to_csv(f, index=False)

In [22]:
# ---- salvar Gold ----
with fs.open(GOLD_SEASON_STATS, "wb") as f:
    stats.to_csv(f, index=False)

print("Gold (season_team_stats) salva em:", GOLD_SEASON_STATS)
stats.head()

Gold (season_team_stats) salva em: gs://meu-bucket-premier/gold/epl/season_team_stats/season_team_stats.csv


,Season,Team,Jogos_Casa,Vitorias_Casa,Empates_Casa,Derrotas_Casa,Gols_Marc_Casa,Gols_Sof_Casa,Jogos_Fora,Vitorias_Fora,...,Jogos,Vitorias,Empates,Derrotas,Gols_Marc,Gols_Sof,Saldo_Gols,Pontos,Team_CNPJ_Masked,_processing_ts
0,2014-2015,CHELSEA,19,15,4,0,36,9,19,11,...,38,26,9,3,73,32,41,87,NaN,2025-08-13T18:32:36+00:00
1,2014-2015,MAN CITY,19,14,3,2,44,14,19,10,...,38,24,7,7,83,38,45,79,NaN,2025-08-13T18:32:36+00:00
2,2014-2015,ARSENAL,19,12,5,2,41,14,19,10,...,38,22,9,7,71,36,35,75,NaN,2025-08-13T18:32:36+00:00
3,2014-2015,MAN UTD,19,14,2,3,41,15,19,6,...,38,20,10,8,62,37,25,70,NaN,2025-08-13T18:32:36+00:00
4,2014-2015,TOTTENHAM,19,10,3,6,31,24,19,9,...,38,19,7,12,58,53,5,64,NaN,2025-08-13T18:32:36+00:00
